# Kara One - Transduction

## Load Dataset

In [1]:
from lib import KaraOneDataset

dataset_dir = "C:\\Users\\win8t\\OneDrive\\Desktop\\projects\\kara-one-transduction\\p"
dataset = KaraOneDataset(dataset_dir)

In [2]:
dataset[0]

True
16000
PATH: C:\Users\win8t\OneDrive\Desktop\projects\kara-one-transduction\p\spoclab\users\szhao\EEG\data\MM05\Acquisition 232 Data.cnt
Reading 0 ... 2477399  =      0.000 ...  2477.399 secs...


{'label': '/uw/',
 'audio': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.05175781e-05, 0.00000000e+00]),
 'emg': array([[-0.02040429, -0.0204033 , -0.02040707, ..., -0.01750357,
         -0.01751409, -0.01750607],
        [-0.01680895, -0.01681136, -0.01681457, ..., -0.0179836 ,
         -0.01799233, -0.01798686],
        [-0.0252607 , -0.02526095, -0.02526514, ..., -0.02211011,
         -0.02211827, -0.02211488],
        ...,
        [-0.00265201, -0.00265287, -0.00265266, ..., -0.01152627,
         -0.01152911, -0.01152572],
        [ 0.00302711,  0.00302566,  0.00302705, ...,  0.00323691,
          0.00323423,  0.00323722],
        [-0.01240757, -0.01240912, -0.01240977, ..., -0.01242258,
         -0.01242561, -0.01242265]])}